In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = False

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(max_depth=3, max_leaf_nodes=336, min_samples_leaf=0.01,\n                       random_state=75, splitter='random')",
  'RandomForestClassifier(max_depth=10, min_samples_leaf=0.0099, n_jobs=-1,\n                       random_state=75)',
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',


In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[250:]
odds_train = odds_df[250:]
label_train = label_df[250:]

df_test = df[:250]
odds_test = odds_df[:250]
label_test = label_df[:250]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

250
250
250
4483
4483
4483


<ipython-input-5-d6c2a1ffdd58>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-5-d6c2a1ffdd58>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [6]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


250
4483


In [7]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
len(models[0])

15

In [9]:
score_list = []
for model_num in range(len(models[0])):
    test_model_name = models[0][model_num]
    test_model = eval(models[1][model_num])
    test_model_features = eval(models[2][model_num])
    test_model_ev = eval(models[3][model_num])
    
    score_list.append(evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True))

(4483, 9)

(4483,)

(4483, 2)

(250, 9)

(250,)

...

(248, 26)

(248,)

(248, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-118,-106,0.546551,0.453449,0
1,140,-177,0.460358,0.539642,1
2,-150,120,0.637225,0.362775,0
3,105,-134,0.455550,0.544450,0
4,-177,140,0.600822,0.399178,1
...,...,...,...,...,...
243,-200,160,0.753626,0.246374,0
244,-335,250,0.769302,0.230698,0
245,-155,125,0.595036,0.404964,1
246,-286,225,0.712245,0.287755,0



          Number of matches: 248
          Number of bets: 152
          Number of winning bets: 69
          Number of losing bets: 83
          Number of underdog bets: 106
          Number of underdog wins: 39
          Number of underdog losses: 67
          Number of Favorite bets: 45
          Number of favorite wins: 29
          Number of favorite losses: 16
          Number of even bets: 1
          Number of even wins: 1
          Number of even losses: 0
          Profit: -296.0965472590791
          Profit per bet: -1.9480036003886783
          Profit per match: -1.1939376905608028
          
          


In [10]:
print(models[4])

['538.4166865', '-324.7349695106435', '-949.3347427574704', '1205.0828045280437', '79.82596213788835', '1165.977716', '528.6527731', '1500.93994', '493.1647765', '-431.4817089', '-35.23037836', '-19.43318131', '290.9104159', '-91.0629001', '-1129.80692']


In [11]:
print(score_list)

[1751.8798456314714, 32.397091539861236, -580.852509498543, -364.5488554206099, -28.17016060750852, 1650.1583341762712, 468.0115554029955, 2570.446418422199, -513.9756547885817, 140.33314097840008, 287.0186563961137, 703.5510631112703, -126.63792775981362, 622.5142487140188, -296.0965472590791]


In [12]:
models[4] = score_list

In [13]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(max_depth=3, max_leaf_nodes=336, min_samples_leaf=0.01,\n                       random_state=75, splitter='random')",
  'RandomForestClassifier(max_depth=10, min_samples_leaf=0.0099, n_jobs=-1,\n                       random_state=75)',
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB(var_smoothing=1e-08)',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)',
  'GaussianNB()',
  'GaussianNB()',
  'GradientBoostingClassifier(n_estimators=122, random_state=75)',


In [14]:
with open('models.csv', 'w' , newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI
